In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Credit_Card'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pathToFolder = "/content/drive/MyDrive/Credit_Card.zip"

In [5]:
path =  "/content/drive/MyDrive/Credit_Card/application_record.csv"

In [6]:
df_app = pd.read_csv(path)
df_app.head(3)


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0


In [7]:
df_cred = pd.read_csv('/content/drive/MyDrive/Credit_Card/credit_record.csv')
df_cred.head(3)

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0


In [8]:
final_df = df_app.merge(df_cred, on=['ID'],how='inner')
final_df.head()
# Merge both application_record and credit record on ID

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0,C
1,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-1,C
2,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-2,C
3,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-3,C
4,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-4,C


In [9]:
final_df.isnull().sum()

ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_MOBIL                  0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        240048
CNT_FAM_MEMBERS             0
MONTHS_BALANCE              0
STATUS                      0
dtype: int64

In [10]:
final_df.shape

(777715, 20)

In [11]:
# drop null values, axis=0
final_df.dropna(inplace=True)
final_df.shape

(537667, 20)

In [12]:
#1,2,3,4 and 5 are bad clients, X and C are good clients and 0 consider as neutral (not bad and not good) so let's take 0 also as good client.
# replacing the status values as good -> 1, bad -> 0

final_df.STATUS = final_df.STATUS.map({"X":1,"C":1, "0":1, "1":0, "2":0, "3":0, "4":0, "5":0})
final_df.sample(5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
745511,5148852,F,Y,N,2,135000.0,Working,Higher education,Married,House / apartment,-12804,-4977,1,0,0,0,Accountants,4.0,-51,1
406194,5079255,F,Y,Y,0,162000.0,Working,Higher education,Separated,House / apartment,-15652,-3404,1,1,1,0,Accountants,1.0,-18,1
480707,5092457,M,N,N,1,247500.0,Working,Secondary / secondary special,Separated,Municipal apartment,-16107,-5313,1,0,1,0,High skill tech staff,2.0,-29,1
618677,5117069,F,Y,Y,2,121500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-12797,-249,1,0,0,0,High skill tech staff,4.0,-6,1
624615,5117502,M,N,Y,0,90000.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-16459,-704,1,1,0,0,Low-skill Laborers,2.0,-6,1


In [13]:
final_df.isnull().sum()

ID                     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
DAYS_BIRTH             0
DAYS_EMPLOYED          0
FLAG_MOBIL             0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
MONTHS_BALANCE         0
STATUS                 0
dtype: int64

In [14]:
final_df.head(4)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
31,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0,1
32,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-1,1
33,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-2,1
34,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-3,1


In [15]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
# replace M & F as 0 & 1
final_df['CODE_GENDER'].replace('M',0,inplace=True) #male -> 0
final_df['CODE_GENDER'].replace('F',1,inplace=True)#female -> 1
final_df['FLAG_OWN_CAR'].replace('Y',0,inplace=True)
final_df['FLAG_OWN_CAR'].replace('N',1,inplace=True)
final_df['FLAG_OWN_REALTY'].replace('Y',0,inplace=True)
final_df['FLAG_OWN_REALTY'].replace('N',1,inplace=True)
final_df.head(4)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
31,5008806,0,0,0,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0,1
32,5008806,0,0,0,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-1,1
33,5008806,0,0,0,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-2,1
34,5008806,0,0,0,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-3,1


In [17]:
final_df.NAME_INCOME_TYPE.value_counts()

Working                 332868
Commercial associate    151412
State servant            52733
Pensioner                  332
Student                    322
Name: NAME_INCOME_TYPE, dtype: int64

In [18]:
def income_tans(df,col):
    result_col = []
    for i in df[col]:
        if i == 'Working':
            result_col.append('Working')
        elif i == 'Commercial associate':
            result_col.append('Commercial associate')
        elif i == 'State servant':
            result_col.append('State servant')
        else:
            result_col.append('others')
    df[col] = result_col
    return df


income_tans(final_df,'NAME_INCOME_TYPE')
final_df.NAME_INCOME_TYPE.value_counts()

Working                 332868
Commercial associate    151412
State servant            52733
others                     654
Name: NAME_INCOME_TYPE, dtype: int64

In [19]:
final_df.NAME_EDUCATION_TYPE.value_counts()

Secondary / secondary special    358317
Higher education                 153770
Incomplete higher                 20590
Lower secondary                    4556
Academic degree                     434
Name: NAME_EDUCATION_TYPE, dtype: int64

In [20]:
final_df.NAME_EDUCATION_TYPE = final_df.NAME_EDUCATION_TYPE.str.split('/').str[0]
final_df.NAME_EDUCATION_TYPE.value_counts()

Secondary            358317
Higher education     153770
Incomplete higher     20590
Lower secondary        4556
Academic degree         434
Name: NAME_EDUCATION_TYPE, dtype: int64

In [21]:
final_df.NAME_FAMILY_STATUS.value_counts()

Married                 384003
Single / not married     65944
Civil marriage           44083
Separated                31394
Widow                    12243
Name: NAME_FAMILY_STATUS, dtype: int64

In [22]:
final_df.NAME_FAMILY_STATUS = final_df.NAME_FAMILY_STATUS.str.split('/').str[0]
final_df.NAME_FAMILY_STATUS.value_counts()

Married           384003
Single             65944
Civil marriage     44083
Separated          31394
Widow              12243
Name: NAME_FAMILY_STATUS, dtype: int64

In [23]:
final_df.head(3)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
31,5008806,0,0,0,0,112500.0,Working,Secondary,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0,1
32,5008806,0,0,0,0,112500.0,Working,Secondary,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-1,1
33,5008806,0,0,0,0,112500.0,Working,Secondary,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-2,1


In [24]:
final_df.AMT_INCOME_TOTAL = final_df.AMT_INCOME_TOTAL/1000
final_df.AMT_INCOME_TOTAL = final_df.AMT_INCOME_TOTAL.astype('int')
final_df.head(3)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
31,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0,1
32,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-1,1
33,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-2,1


In [25]:
final_df['Age']=-(final_df['DAYS_BIRTH'])//365

final_df['employee_from_years']=-(final_df['DAYS_EMPLOYED'])//365

final_df.drop(columns=('DAYS_BIRTH'), inplace=True)

final_df.drop(columns=('DAYS_EMPLOYED'), inplace=True)
final_df.head(4)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS,Age,employee_from_years
31,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,0,1,58,3
32,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,-1,1,58,3
33,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,-2,1,58,3
34,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,-3,1,58,3


In [26]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 537667 entries, 31 to 777714
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   537667 non-null  int64  
 1   CODE_GENDER          537667 non-null  int64  
 2   FLAG_OWN_CAR         537667 non-null  int64  
 3   FLAG_OWN_REALTY      537667 non-null  int64  
 4   CNT_CHILDREN         537667 non-null  int64  
 5   AMT_INCOME_TOTAL     537667 non-null  int64  
 6   NAME_INCOME_TYPE     537667 non-null  object 
 7   NAME_EDUCATION_TYPE  537667 non-null  object 
 8   NAME_FAMILY_STATUS   537667 non-null  object 
 9   NAME_HOUSING_TYPE    537667 non-null  object 
 10  FLAG_MOBIL           537667 non-null  int64  
 11  FLAG_WORK_PHONE      537667 non-null  int64  
 12  FLAG_PHONE           537667 non-null  int64  
 13  FLAG_EMAIL           537667 non-null  int64  
 14  OCCUPATION_TYPE      537667 non-null  object 
 15  CNT_FAM_MEMBERS 

In [27]:
final_df.head(4)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS,Age,employee_from_years
31,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,0,1,58,3
32,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,-1,1,58,3
33,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,-2,1,58,3
34,5008806,0,0,0,0,112,Working,Secondary,Married,House / apartment,1,0,0,0,Security staff,2.0,-3,1,58,3


In [28]:
final_df.drop(columns=['ID','CNT_CHILDREN','FLAG_MOBIL','FLAG_WORK_PHONE','FLAG_EMAIL'], inplace=True)
final_df.sample(10)

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,FLAG_PHONE,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS,Age,employee_from_years
133504,1,1,0,315,State servant,Secondary,Civil marriage,House / apartment,0,Core staff,3.0,-43,1,30,6
654436,0,0,0,112,Working,Secondary,Married,House / apartment,0,Medicine staff,4.0,-2,1,27,5
448508,1,1,0,175,Commercial associate,Higher education,Married,House / apartment,1,Sales staff,4.0,-17,1,46,3
237757,0,0,0,180,Working,Secondary,Civil marriage,House / apartment,0,Laborers,4.0,-22,1,35,13
49618,1,1,0,292,Commercial associate,Higher education,Civil marriage,House / apartment,0,Laborers,2.0,-11,1,39,4
388858,1,1,0,135,Working,Secondary,Single,House / apartment,1,Secretaries,1.0,-11,1,62,11
304218,1,1,0,324,State servant,Higher education,Married,House / apartment,0,Managers,3.0,-12,1,36,13
88248,1,1,0,112,Working,Secondary,Separated,House / apartment,0,Sales staff,3.0,-11,1,39,9
536065,0,0,0,180,Working,Secondary,Married,House / apartment,0,Drivers,2.0,-18,1,59,6
6223,1,1,1,292,State servant,Secondary,Married,House / apartment,1,Accountants,2.0,-37,1,57,23


In [29]:
# one hot encoding
def one_hot_encode(df, columns):
    for col in columns:
        one_hot = pd.get_dummies(df[col], drop_first=True)
        df = pd.concat([df, one_hot], axis=1)
        df = df.drop([col], axis=1)
    return df

categories=['NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE']
final_df = one_hot_encode(final_df,categories)
final_df.head(5)

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,FLAG_PHONE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS,Age,employee_from_years,...,Laborers,Low-skill Laborers,Managers,Medicine staff,Private service staff,Realty agents,Sales staff,Secretaries,Security staff,Waiters/barmen staff
31,0,0,0,112,0,2.0,0,1,58,3,...,0,0,0,0,0,0,0,0,1,0
32,0,0,0,112,0,2.0,-1,1,58,3,...,0,0,0,0,0,0,0,0,1,0
33,0,0,0,112,0,2.0,-2,1,58,3,...,0,0,0,0,0,0,0,0,1,0
34,0,0,0,112,0,2.0,-3,1,58,3,...,0,0,0,0,0,0,0,0,1,0
35,0,0,0,112,0,2.0,-4,1,58,3,...,0,0,0,0,0,0,0,0,1,0


In [30]:
X = final_df.drop(columns=('STATUS'),axis=1)
y = final_df.STATUS
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)
print('X_train :', X_train.shape)
print('X_test :', X_test.shape)
print('y_train :', y_train.shape)
print('y_test :', y_test.shape)

X_train : (430133, 42)
X_test : (107534, 42)
y_train : (430133,)
y_test : (107534,)


In [33]:
ss = StandardScaler()

In [34]:
ss_fit = ss.fit_transform(X)

In [35]:
X_train,X_test, y_train,y_test = train_test_split(X, y, test_size = 0.25)

In [36]:
X_train.shape
X_test.shape

(134417, 42)

In [37]:
mm = MinMaxScaler()
mm.fit(X_train)

MinMaxScaler()

In [38]:
model = tf.keras.Sequential()

In [39]:
model.add(layers.Input( shape = (42,) ) )

In [40]:
model.add(tf.keras.layers.Dense(500, activation = 'relu'))

In [41]:
model.add(tf.keras.layers.Dense(2, activation = 'relu'))

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               21500     
                                                                 
 dense_1 (Dense)             (None, 2)                 1002      
                                                                 
Total params: 22,502
Trainable params: 22,502
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.compile(loss = "mae")

In [44]:
history1 = model.fit(X_train,y_train,
                    epochs = 70,
                    validation_data = (X_test, y_test)
                   )

Epoch 1/70
12602/12602 [==============================] - 23s 2ms/step - loss: 0.9848 - val_loss: 0.9844
Epoch 2/70
12602/12602 [==============================] - 22s 2ms/step - loss: 0.9844 - val_loss: 0.9844
Epoch 3/70
12602/12602 [==============================] - 23s 2ms/step - loss: 0.9844 - val_loss: 0.9844
Epoch 4/70
12602/12602 [==============================] - 23s 2ms/step - loss: 0.9844 - val_loss: 0.9844
Epoch 5/70
12602/12602 [==============================] - 22s 2ms/step - loss: 0.9844 - val_loss: 0.9844
Epoch 6/70
12602/12602 [==============================] - 22s 2ms/step - loss: 0.9844 - val_loss: 0.9844
Epoch 7/70
12602/12602 [==============================] - 21s 2ms/step - loss: 0.9844 - val_loss: 0.9844
Epoch 8/70
12602/12602 [==============================] - 22s 2ms/step - loss: 0.9844 - val_loss: 0.9844
Epoch 9/70
12602/12602 [==============================] - 22s 2ms/step - loss: 0.9844 - val_loss: 0.9844
Epoch 10/70
12602/12602 [==============================

In [46]:
model.evaluate(X_test,y_test)

4201/4201 [==============================] - 4s 897us/step - loss: 0.9844


0.9844439029693604